<a href="https://colab.research.google.com/github/ManasaMAY19/AI-chatbot-with-NLP/blob/main/Sales_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SALES ANALYSIS

Hi! This is my Sales Analysis notebook for [Sales Product Data](https://www.kaggle.com/datasets/knightbearr/sales-product-data) from [Knightbearr](https://www.kaggle.com/knightbearr). The purpose is to provide analysis based on dataset provided by going through all the necessary step and methods in this notebook alongside its explanation. Since there is already task provided in the dataset description making it easier to explore different means to answer multiple questions and provide analysis. Also note that I'm still learning and there may be mistake and few things i might be overlooked. If that happen feel free comment and point them out.

Thanks!

# Table of Contents
[SALES ANALYSIS](#SALES-ANALYSIS)
- [LIBRARY IMPORT](#LIBRARY-IMPORT)
- [DATA READING](#DATA-READING)
- [DATA CLEANING](#DATA-CLEANING)
- [DATA MANIPULATION](#DATA-MANIPULATION)
- [QUESTIONS](#QUESTIONS)
    - [Total Sales](#Total-Sales)
        - [Annual Sales](#Annual-Sales)
        - [Monthly Sales](#Monthly-Sales)
    - [Sales by State & City](#Sales-by-State-and-City)
        - [Sales by State](#Sales-by-State)
        - [Sales by City](#Sales-by-City)
    - [Order Trend](#Order-Trend)
        - [Monthly Order Trend](#Monthly-Order-Trend)
        - [Daily Order Trend](#Daily-Order-Trend)
        - [Hourly Order Trend](#Hourly-Order-Trend)
    - [Product Performance](#Product-Performance)
        - [Product Popularity](#Product-Popularity)
        - [Price List Comparision](#Price-List-Comparision)
        - [Product Sold by Order](#Product-Sold-by-Order)
    - [Probability](#Probability)
        - [Product Orders of Total Sales](#Product-Orders-of-Total-Sales)
    - [Product Associate Rules](#Product-Associate-Rules)
- [SUMMARY](#TO-SUMMARIZE)
- [REFERENCES](#REFERENCES)
- [VERSION](#VERSION-12)

# LIBRARY IMPORT

> Import all required library for files import/export, plotting, and association rules.

In [ ]:
#import required library
import os
import squarify
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display_html

#required library for one-hot matrix and association rules
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules


# DATA READING

> Combine all related CSV data into a single dataframe(all_sales_df) then rename and lowercase all the column name.

In [ ]:
#find file path directory
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
#data path
dir_path = '../input/sales-product-data'

#create empty df for all sales
all_sales_df = pd.DataFrame()

#concat all csv in dir path
for filename in os.listdir(dir_path):
    df = pd.read_csv(os.path.join(dir_path, filename))
    all_sales_df = pd.concat ([all_sales_df, df], ignore_index=True)

#rename column name
all_sales_df = all_sales_df.rename(mapper = str.strip, axis='columns')
all_sales_df = all_sales_df.rename(columns={'Order ID': 'Order_id',
                                            'Quantity Ordered': 'Quantity',
                                            'Price Each': 'Price',
                                            'Order Date': 'Date',
                                            'Purchase Address': 'Address'})

#lowercase column name
column_name = list(all_sales_df.columns)
column_name = [x.lower().strip() for x in column_name]
all_sales_df.columns = column_name

#show dataframe info
print(all_sales_df.info())


In [ ]:
#show all_sales_df head
all_sales_df.head()


# DATA CLEANING

> First check for rows contain null and duplicates.
Then check for non-numeric values in order_id, quantity, and price since the 3 column cannot be non-numeric.

In [ ]:
#check for all rows contain null value
print('rows contain null:')
print(all_sales_df.isnull().sum())

#check duplicated rows
print('rows contain duplicates:', all_sales_df.duplicated().sum())

#check for non-numeric in order_id, quantity, and price
print('non-numeric in order_id:', all_sales_df['order_id'].loc[pd.to_numeric(all_sales_df['order_id'], errors='coerce').isnull()].unique())
print('non-numeric in quantity:', all_sales_df['quantity'].loc[pd.to_numeric(all_sales_df['quantity'], errors='coerce').isnull()].unique())
print('non-numeric in price:', all_sales_df['price'].loc[pd.to_numeric(all_sales_df['price'], errors='coerce').isnull()].unique())


> There is a few consideration about dropping duplicated rows.
Check duplicated rows and their duplicates.

In [ ]:
#check order id in duplicated rows that are not null and is numeric
all_sales_df[all_sales_df.duplicated(keep=False) & all_sales_df['order_id'].notnull() & all_sales_df['order_id'].str.isnumeric()]['order_id'].head()


In [ ]:
#check rows contain order id of 176585, 177795, 178158
all_sales_df[all_sales_df['order_id'].isin(['176585', '177795', '178158'])]


>Now since order id 176585, 177795, and 178158 despite having quantity column it still have more than 1 of same rows each.
Which doesn't make sense when the product is the same it should be added into 'quantity' instead of adding another duplicate row. Another thing to consider is that we can join the rows from duplicates into rows that are duplicated and add the value into quantity.
But since it is still unclear, the next cells will continue to remove the duplicate rows.


> If there is null and duplicated rows then it need to be dropped/removed.
For non-numeric in mentioned columns, apply to_numeric to remove the rows dataframe.
Then check the result by applying the previous methods for a new defined dataframe(clean_sales_df).

In [ ]:
#drop null rows
clean_sales_df = all_sales_df.dropna(how='all')

#drop all duplicates rows
clean_sales_df = clean_sales_df[~ clean_sales_df.duplicated()]

#drop non-numeric values in order_id, quantity, and price
clean_sales_df = clean_sales_df[pd.to_numeric(clean_sales_df['order_id'], errors='coerce').notnull()]
clean_sales_df = clean_sales_df[pd.to_numeric(clean_sales_df['quantity'], errors='coerce').notnull()]
clean_sales_df = clean_sales_df[pd.to_numeric(clean_sales_df['price'], errors='coerce').notnull()]

#print data cleaning result
#null result
print('rows contain null:')
print(clean_sales_df.isnull().sum())

#check duplicated rows
print('rows contain duplicates:', clean_sales_df.duplicated().sum())

#unique value result
print('non-numeric in order_id:', clean_sales_df['order_id'].loc[pd.to_numeric(clean_sales_df['order_id'], errors='coerce').isnull()].unique())
print('non-numeric in quantity:', clean_sales_df['quantity'].loc[pd.to_numeric(clean_sales_df['quantity'], errors='coerce').isnull()].unique())
print('non-numeric in price:', clean_sales_df['price'].loc[pd.to_numeric(clean_sales_df['price'], errors='coerce').isnull()].unique())


# DATA MANIPULATION

> Before analysis it's important to define data type for each column that will be called later on.
Split city and state from 'address' into separate 'city' and 'state' column which will be required later for visualizations.
Add 'total_sales' column from 'quantity'*'price'.

In [ ]:
#change data type for column quantity, price, date, and address
clean_sales_df['quantity'] = clean_sales_df['quantity'].astype(int)
clean_sales_df['price'] = clean_sales_df['price'].astype(float)
clean_sales_df['date'] = pd.to_datetime(clean_sales_df['date'], format='%m/%d/%y %H:%M')
clean_sales_df['address'] = clean_sales_df['address'].astype(str)

#add city and state column
clean_sales_df['city'] = clean_sales_df['address'].apply(lambda x: x.split(',')[1].strip())
clean_sales_df['state'] = clean_sales_df['address'].apply(lambda x: x.split(',')[2].split(' ')[1].strip())

#add total_sales column
clean_sales_df['total_sales'] = clean_sales_df['quantity'] * clean_sales_df['price']


> From here since data outside year 2019 is less significant.
The following try to separate clean_sales_df(all data) and clean_sales_2019_df(2019 data).
But we can check how many rows contain data outside 2019 by applying following method.

In [ ]:
#find data outside 2019
print('number of rows outside 2019:', len(clean_sales_df[clean_sales_df['date'].dt.year != 2019]))


In [ ]:
#store 2019 data in separate dataframe
clean_sales_2019_df = pd.DataFrame(clean_sales_df[clean_sales_df['date'].dt.year == 2019]) #this will only include rows with year 2019

#print data manipulation result
#2019 data
print('clean_sales_2019 data result:')
print(clean_sales_2019_df.info())

#data outside 2019
print('number of rows outside 2019 in clean_sales_2019_df:', len(clean_sales_2019_df[clean_sales_2019_df['date'].dt.year != 2019]))


> Reset the index for both dataframe and print data head to show dataframe result.

In [ ]:
#reset data index for both dataframe
clean_sales_df = clean_sales_df.reset_index(drop=True)
clean_sales_2019_df = clean_sales_2019_df.reset_index(drop=True)

#check data head before analysis/visualization
clean_sales_df.head()


# QUESTIONS

## Total Sales

Before advancing through [Annual Sales](#Annual-Sales), [Monthly Sales](#Monthly-Sales) and [Sales by State & City](#Sales-by-State-and-City).
The following section/cells is made to define function that will be used specifically for total sales related plot/sections to make the rest of the codes much less repetitive.

> Define function for plotting Annual Sales and Monthly Sales referring to 'total_sales' so it's easier and less repetitive.
In this case the method will refer to pivot type so x or y value may refer to index instead of column name.
If the source is dataframe then refer to column name instead of index.

In [ ]:
#define function for total sales visual and add parameters
def plot_total_sales(pivot, width, height, plotting, fig_title, avg_series=None):
    with plt.style.context('ggplot'):
        #format figure
        plt.figure(figsize=(width, height))
        ax = getattr(sns, plotting)(x=pivot.index, y='total_sales', data=pivot)
        ax.ticklabel_format(style='plain', axis='y')
        ax.set(title=fig_title, xlabel='Date', ylabel='Sales')

        #if statement to add axh line for avg series
        if avg_series is not None:
            ax.axhline(y=avg_series, color='blue', linestyle=':', linewidth=1)
            ax.text(pivot.index[-12], avg_series+5, f'Avg: ${avg_series:,.2f}', color='blue', fontsize=8, ha='center', va='bottom')

        #add annotation
        for x, y in enumerate(pivot['total_sales']):
            label = f'${y:,.2f}'
            ax.annotate(label, (x, y), textcoords='offset points', xytext=(0, 4), ha='center')
        plt.show()


> Defining function(sort_style_total_sales) for sort by value and adding color gradient to pivot table might not be necessary if it's used once.
But the following function will be used later in Monthly Sales and Sales by State/City.

In [ ]:
#define function for pivot sort by value in total sales and return pivot with color gradient
def sort_style_total_sales(pivot, text):
    #sort pivot by value in total sales
    pivot = pivot.sort_values(by='total_sales', ascending=False)

    #format pivot with color gradient
    styled_pivot = pivot.style.background_gradient(cmap='RdYlGn', subset=['total_sales']).format({'total_sales': '${:,.2f}'})

    #pass table to html
    html_pivot = styled_pivot.to_html()

    #display tabel
    display_html(f'CONCLUSION:\n'
                 f'<table><tr><th>{text}</th></tr><tr><td>{html_pivot}</td></tr></table>', raw=True)


> Similar case also goes for function(max_min_total_sales) to find max and min values in total_sales.

In [ ]:
#define function to find highest and lowest total sales for conclusion
def max_min_total_sales(pivot):
    #find highest value
    highest_date = pivot['total_sales'].idxmax()
    highest_value = pivot['total_sales'].max()

    #find lowest value
    lowest_date = pivot['total_sales'].idxmin()
    lowest_value = pivot['total_sales'].min()

    #print result as conclusion
    print(f"CONCLUSION:\n"
          f"Total sales generated by Company: ${pivot['total_sales'].sum():,.2f}\n"
          f"\n"
          f"With the highest sales is in {highest_date}, generating: ${highest_value:,.2f}\n"
          f"With the lowest sales is in {lowest_date}, generating: ${lowest_value:,.2f}")


### Annual Sales

Task 1:

Q: What was the best Year for sales? How much was earned that Year?

A: Create Annual Sales summary and find max/min values in Annual Sales

In this section clean_sales_df will be referred to find annual sales as annual_sales_pivot.
Plot the annual_sales_pivot and find max/min values then show the result by calling the previously defined function.

In [ ]:
#find annual sales
annual_sales_pivot = pd.pivot_table(clean_sales_df, index=clean_sales_df['date'].dt.year, values='total_sales', aggfunc='sum')

#plot annual sales
plot_total_sales(annual_sales_pivot, 4, 4, 'barplot','Annual Sales')


In [ ]:
#sort and add color gradient for annual sales
sort_style_total_sales(annual_sales_pivot, 'Annual Sales:')


In [ ]:
#find max min value in annual sales for conclusion
max_min_total_sales(annual_sales_pivot)


### Monthly Sales

Task 2:

Q: What was the best month for sales? How much was earned that month?

A: Create Monthly Sales summary and find largest/smallest values in Monthly Sales

In this section clean_sales_df will be referred to find monthly sales as monthly_sales_pivot.
Plot and find largest/smallest values in monthly_sales_pivot then show the result conclusion.
since all the method has been defined previously, the following will just call the function.

In [ ]:
#find and pivot monthly sales, then change the index format if necessary
monthly_sales_pivot = pd.pivot_table(clean_sales_df, index=clean_sales_df['date'].dt.to_period('M'), values='total_sales', aggfunc='sum')
monthly_sales_pivot.index = monthly_sales_pivot.index.strftime('%b %Y')

#find average of total sales in monthly sales for axh line
average_monthly_sales = monthly_sales_pivot['total_sales'].mean()

#plot monthly sales and pass the average series
plot_total_sales(monthly_sales_pivot, 16, 4, 'lineplot', 'Monthly Sales', average_monthly_sales)


In [ ]:
#sort and add color gradient for monthly sales
sort_style_total_sales(monthly_sales_pivot, 'Monthly Sales:')


In [ ]:
#find max and min value in monthly sales for conclusion
max_min_total_sales(monthly_sales_pivot)


## Sales by State and City

Task 3:

Q: What City had the highest number of sales?

A: Create Sales by State/City and find largest/smallest values in Sales by State/City.

This section separated by 2 for [Sales by State](###Sales-by-State) and [Sales by City](###Sales-by-City) from clean_sales_df.
Purpose is to find both trend on state level and city level.
Plot and find largest/smallest values for both of the result.

> The following function(large_small_total_sales) is used to find 3 largest and smallest which is different from previous function(max_min_total_sales). Another difference is that instead of printing the result as string the following will show pivot as side by side html with color gradient applied if needed.

In [ ]:
#create function to find 3 largest and 3 smallest value then show the result as html
def large_small_total_sales(pivot, label_1, label_2):
    #find 3 largest and smallest total sales
    largest = pivot.nlargest(3, 'total_sales')
    smallest = pivot.nsmallest(3, 'total_sales')

    #format table with gradient
    largest_styled = largest.style.background_gradient(cmap='RdYlGn', subset=['total_sales']).format({'total_sales': '${:,.2f}'})
    smallest_styled = smallest.style.background_gradient(cmap='RdYlGn', subset=['total_sales']).format({'total_sales': '${:,.2f}'})

    #pass table to html
    largest_html = largest_styled.to_html()
    smallest_html = smallest_styled.to_html()

    #display tabel
    display_html(f'CONCLUSION:\n'
                 f'<table><tr><th>{label_1}</th><th>{label_2}</th></tr><tr><td>{largest_html}</td><td>{smallest_html}</td></tr></table>', raw=True)


### Sales by State

In [ ]:
#find and pivot sales by state then sort the values from highest to lowest
sales_by_state_pivot = pd.pivot_table(clean_sales_df, index='state', values='total_sales', aggfunc=sum)
sales_by_state_pivot = sales_by_state_pivot.sort_values(by='total_sales', ascending=False)


In [ ]:
#generate state color and store it in dictionary for reuse
state_color = sns.color_palette('husl', len(sales_by_state_pivot.index.unique()))
state_color_dict = dict(zip(sales_by_state_pivot.index.unique(), state_color))

#plot sales by state
with plt.style.context('fivethirtyeight'):
    #format figure
    plt.figure(figsize=(16, 4))
    ax = sns.barplot(x=sales_by_state_pivot.index, y='total_sales', data=sales_by_state_pivot, palette=state_color_dict)
    ax.set(title='Sales by State', xlabel='State', ylabel='Sales')
    ax.ticklabel_format(style='plain', axis='y')

    #add data annotation
    for x, y in enumerate(sales_by_state_pivot['total_sales']):
        label = f'${y:,.2f}'
        ax.annotate(label, (x, y), textcoords='offset points', xytext=(0, 2), ha='center', fontsize=10)
    plt.show()


In [ ]:
#sort and show the pivot with gradient
sort_style_total_sales(sales_by_state_pivot, 'Sales by State:')


In [ ]:
#find 3 largest and smallest in state sales
large_small_total_sales(sales_by_state_pivot, 'Largest State Sales:', 'Smallest State Sales:')


### Sales by City

> There is a problem when referring y='city' in following cell for a same city name Portland, Maine and Portland, Oregon. Which in return overwrite the y-axis.
The solution is to rename the city name for 'Portland' by referring 'state' then return with new defined names.

In [ ]:
#find and pivot sales by city
sales_by_city_pivot = pd.pivot_table(clean_sales_df, values='total_sales', index=['state', 'city'], aggfunc='sum')
sales_by_city_pivot = sales_by_city_pivot.sort_values(by='total_sales', ascending=False).reset_index()

#modify city named 'Portland' before generating visual (to avoid overwrite)
sales_by_city_pivot.loc[(sales_by_city_pivot['state'] == 'ME') & (sales_by_city_pivot['city'] == 'Portland'), 'city'] = 'Portland (Maine)'
sales_by_city_pivot.loc[(sales_by_city_pivot['state'] == 'OR') & (sales_by_city_pivot['city'] == 'Portland'), 'city'] = 'Portland (Oregon)'


In [ ]:
#plot sales by city
with plt.style.context('fivethirtyeight'):
    #format figure size
    plt.figure(figsize=(16, 4))

    #format figure
    ax = sns.barplot(x='total_sales', y='city', hue='state', data=sales_by_city_pivot, dodge=False, palette=state_color_dict)
    ax.set(title='Sales by City', xlabel='Sales', ylabel='City')
    ax.ticklabel_format(style='plain', axis='x')
    ax.legend(title='State', bbox_to_anchor=(1.05, 0.5), loc='center left', borderaxespad=0.)

    #add data annotation
    for y, x in enumerate(sales_by_city_pivot['total_sales']):
        label = f'${x:,.2f}'
        ax.annotate(label, (x, y), textcoords='offset points', xytext=(0, 0), ha='left', va='center')
    plt.show()


In [ ]:
#add color gradient for sales by city and set index as state and city
sort_style_total_sales(sales_by_city_pivot.set_index(['state', 'city']), 'Sales by City:')


In [ ]:
#call the function to display result and set index as state and city
large_small_total_sales(sales_by_city_pivot.set_index(['state', 'city']), 'Largest City Sales:', 'Smallest City Sales:')


## Order Trend

Task 4:

Q: What time should we display adverstisement to maximize likelihood of customer's buying product?

A: Find Monthly/Daily/Hourly Order Trend

This section will find the data related to orders by simply find count/unique number of order_id by month/day/hour.
The data that will be referred here is clean_sales_2019_df(which also apply for the rest of the analysis).
Despite order id cannot be duplicate in the dataset there is multiple rows with the same order id.
The reason why it's not dropped is because each may contains different product.

*not sure how to approach/may also need to change [Data Cleaning](#DATA-CLEANING) section

But using 'count' it will return the duplicated order id as well.
If wanted to count the order id as unique number then call 'nunique' which will be used in following sections.

> The following plot function is to plot monthly order, daily order, and hourly order.

In [ ]:
#define function for order trends visual
def plot_trend_total_orders(pivot, plotting, fig_title, avg_series=None, text_pos=None):
    with plt.style.context('bmh'):
        #format figure
        plt.figure(figsize=(16, 4))
        ax = getattr(sns, plotting)(x=pivot.index, y='total_orders', data=pivot)
        ax.ticklabel_format(style='plain', axis='y')
        ax.set(title=fig_title, xlabel='Date/Time', ylabel='Order')
        ax.tick_params(axis='both', labelsize=10)

        #if statement to add axh line for avg series
        if avg_series is not None:
            ax.axhline(y=avg_series, color='red', linestyle=':', linewidth=1)
            ax.text(pivot.index[text_pos], avg_series+5, f'Avg: {avg_series:,.2f}', color='blue', fontsize=8, ha='center', va='bottom')

        #add annotation
        for x, y in enumerate(pivot['total_orders']):
            label = f'{y:,}'
            ax.annotate(label, (x, y), textcoords='offset points', xytext=(0, 4), ha='center', fontsize=10)
        plt.show()


> The following sort_style_total_orders is different from previous sort_style_total_sales since it refer to column name 'total_orders' and have different string format for the values.

In [ ]:
#define function for pivot sort by value and color gradient
def sort_style_total_orders(pivot, text):
    #sort pivot by value in total sales
    pivot = pivot.sort_values(by='total_orders', ascending=False)

    #format as the styled pivot
    styled_pivot = pivot.style.background_gradient(cmap='RdYlGn', subset=['total_orders']).format({'total_orders': '{:,}'})

    #pass table to html
    html_pivot = styled_pivot.to_html()

    #display tabel
    display_html(f'CONCLUSION:\n'
                 f'<table><tr><th>{text}</th></tr><tr><td>{html_pivot}</td></tr></table>', raw=True)


> Add function to find largest and smallest values in 'total_orders' and added size parameter to specify how many value returned.

In [ ]:
#create function to find largest and smallest value in total orders and convert to html
def large_small_total_orders(pivot, label_1, label_2, size):
    #find 3 largest and smallest total orders
    largest = pivot.nlargest(size, 'total_orders')
    smallest = pivot.nsmallest(size, 'total_orders')

    #format table with gradient
    largest_styled = largest.style.background_gradient(cmap='RdYlGn', subset=['total_orders']).format({'total_sales': '{:,}'})
    smallest_styled = smallest.style.background_gradient(cmap='RdYlGn', subset=['total_orders']).format({'total_sales': '{:,}'})

    #pass table to html
    largest_html = largest_styled.to_html()
    smallest_html = smallest_styled.to_html()

    #display tabel
    display_html(f'CONCLUSION:\n'
                 f'<table><tr><th>{label_1}</th><th>{label_2}</th></tr><tr><td>{largest_html}</td><td>{smallest_html}</td></tr></table>', raw=True)


### Monthly Order Trend

In [ ]:
#find and pivot monthly order
monthly_order_pivot = pd.pivot_table(clean_sales_2019_df, index=clean_sales_2019_df['date'].dt.to_period('M'), values='order_id', aggfunc='nunique')

#rename column and format date index if needed
monthly_order_pivot = monthly_order_pivot.rename(columns={'order_id': 'total_orders'})
monthly_order_pivot.index = monthly_order_pivot.index.strftime('%b %Y')
monthly_order_pivot.index.name = 'month'

#find average in monthly order
average_monthly_order = monthly_order_pivot['total_orders'].mean()

#plot monthly order trend and pass the average series and their position(refer to x as index)
plot_trend_total_orders(monthly_order_pivot, 'lineplot', 'Monthly Order Trend', average_monthly_order, -12)


In [ ]:
#sort and add color gradient for monthly order pivot
sort_style_total_orders(monthly_order_pivot, 'Monthly Orders:')


In [ ]:
#find largest and lowest value in monthly order and specify the size returned
large_small_total_orders(monthly_order_pivot, 'Largest Month Orders:', 'Smallest Month Orders:', 3)


### Daily Order Trend

In [ ]:
#find and pivot daily order
daily_order_pivot = pd.pivot_table(clean_sales_2019_df, index=clean_sales_2019_df['date'].dt.dayofweek, values='order_id', aggfunc='nunique')

#rename column and format date index
daily_order_pivot = daily_order_pivot.rename(columns={'order_id': 'total_orders'})
daily_order_pivot.index = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daily_order_pivot.index.name = 'day'

#find average in daily order
average_daily_order = daily_order_pivot['total_orders'].mean()

#plot daily order trend
plot_trend_total_orders(daily_order_pivot, 'lineplot', 'Monthly Order Trend', average_daily_order, -6)


In [ ]:
#sort and add color gradient to daily order pivot
sort_style_total_orders(daily_order_pivot, 'Daily Orders:')


In [ ]:
#find largest and smallest orders in daily order
large_small_total_orders(daily_order_pivot, 'Largest Day Orders:', 'Smallest Day Orders:', 3)


### Hourly Order Trend

In [ ]:
#find and pivot hourly order
hourly_order_pivot = pd.pivot_table(clean_sales_2019_df, index=clean_sales_2019_df['date'].dt.hour, values='order_id', aggfunc='nunique')

#rename column and format date index
hourly_order_pivot = hourly_order_pivot.rename(columns={'order_id': 'total_orders'})
hourly_order_pivot.index = pd.to_datetime(hourly_order_pivot.index, format='%H').strftime('%H:%M')
hourly_order_pivot.index.name = 'hour'

#find average in hourly order
average_hourly_order = hourly_order_pivot['total_orders'].mean()

#plot hourly order trend
plot_trend_total_orders(hourly_order_pivot, 'lineplot', 'Hourly Order Trend', average_hourly_order, -24)


In [ ]:
#sort and add color gradient to hourly order pivot
sort_style_total_orders(hourly_order_pivot, 'Hourly Orders:')


In [ ]:
#find largest and smallest orders in daily order
large_small_total_orders(hourly_order_pivot, 'Largest Hour Order:', 'Smallest Hour Order:', 6)


## Product Performance

Task 5:

Q: What product sold the most? Why do you think it sold the most?

A: Find Products Popularity and Product Price List Comparision

Find data related to Product Popularity by calculate sum of quantity by each product.
This return the total product being purchased.
It also find price list by product which is used to create filled lines for comparision.
However, since revenue(total_sales-total_cost) is not available the following will only use total quantity sold hence "Popularity".

### Product Popularity

In [ ]:
#find and pivot product popularty
product_popularity_pivot = pd.pivot_table(clean_sales_2019_df, index='product', values='quantity', aggfunc='sum')
product_popularity_pivot = product_popularity_pivot.sort_values(by='quantity', ascending=False)

#find and pivot product price
product_price_pivot = pd.pivot_table(clean_sales_2019_df, index='product', values='price', aggfunc='first')

#merge both pivot as one
merged_popularity_price_pivot = pd.merge(product_popularity_pivot, product_price_pivot, on='product')


> The following plot will have x as 'product' with y as quantity(ax1) and price(ax2) to visualize and compare how product perform relative to its price.

In [ ]:
#generate product colors and store it in dictionary for reuse
product_color = sns.color_palette('husl', len(merged_popularity_price_pivot.index.unique()))
product_color_dict = dict(zip(merged_popularity_price_pivot.index.unique(), product_color))

#generate visual for product popularity(merged_popularity_price_pivot)
with plt.style.context('classic'):
    #format figure and ax1
    fig, ax1 = plt.subplots(figsize=(20,6))
    ax1 = sns.barplot(x=merged_popularity_price_pivot.index, y='quantity', data=merged_popularity_price_pivot, palette=product_color_dict)
    ax1.ticklabel_format(style='plain', axis='y')
    ax1.set(title='Product Popularity', xlabel='Product', ylabel='Quantity')
    ax1.set_xticklabels(labels=merged_popularity_price_pivot.index, rotation=80)

    #add data annotation for ax1
    for x, y in enumerate(merged_popularity_price_pivot['quantity']):
        label = f'{y:,}'
        ax1.annotate(label, (x,y), textcoords='offset points', xytext=(0,4), ha='center', fontsize=10)

    #plot product prices as filled line plot as ax2
    ax2 = ax1.twinx()
    ax2.plot(merged_popularity_price_pivot.index, merged_popularity_price_pivot['price'], color='red')
    ax2.set_ylabel('Price')
    ax2.fill_between(merged_popularity_price_pivot.index, merged_popularity_price_pivot['price'], alpha=0.2)
    plt.show()


In [ ]:
#add color gradient and show pivot table
styled_pivot = merged_popularity_price_pivot.style.background_gradient(cmap='RdYlGn', subset=['quantity', 'price']).format({'quantity': '{:,}', 'price': '{:,.2f}'})

#pass table to html
html_pivot = styled_pivot.to_html()

#display tabel
display_html(f'CONCLUSION:\n'
             f'<table><tr><th>Product Popularity Price:</th></tr><tr><td>{html_pivot}</td></tr></table>', raw=True)


### Price List Comparision

In [ ]:
#find price for each item list
product_list_pivot = pd.pivot_table(clean_sales_2019_df, index='product', values='price', aggfunc='first')
product_list_pivot = product_list_pivot.sort_values(by='price', ascending=False)

#find product price average
average_product_price = product_list_pivot['price'].mean()


In [ ]:
#generate product list visual
with plt.style.context('classic'):
    #format figure size
    plt.figure(figsize=(20, 6))

    #format figure
    ax = sns.barplot(x='price', y=product_list_pivot.index, data=product_list_pivot, palette=product_color_dict)
    ax.set(title='Product Price List', xlabel='Price', ylabel='Product')

    #add axv line
    ax.axvline(x=average_product_price, color='red', linestyle='--', linewidth=1)
    ax.text(average_product_price+5, len(product_list_pivot)/2,
            f'Avg: ${average_product_price:.2f}',
            color='red', ha='left', va='center')

    #add data annotation
    for y, x in enumerate(product_list_pivot['price']):
        label = f'${x:,.2f}'
        ax.annotate(label, (x, y), textcoords='offset points', xytext=(0, 0), ha='left', va='center', fontsize=10)
    plt.show()


In [ ]:
#find largest and smallest price in price list
largest = product_list_pivot.nlargest(4, 'price')
smallest = product_list_pivot.nsmallest(4, 'price')

#format table with gradient
largest_styled = largest.style.background_gradient(cmap='RdYlGn', subset=['price']).format({'price': '{:,}'})
smallest_styled = smallest.style.background_gradient(cmap='RdYlGn', subset=['price']).format({'price': '{:,}'})

#pass table to html
largest_html = largest_styled.to_html()
smallest_html = smallest_styled.to_html()

#display tabel
display_html(f'CONCLUSION:\n'
             f'<table><tr><th>Most Expensive Products:</th><th>Most Cheap Products:</th></tr><tr><td>{largest_html}</td><td>{smallest_html}</td></tr></table>', raw=True)


### Product Sold by Order

> Find average item each order and show how likely customers to buy more than 1 product in single order.

In [ ]:
#find average quantity per order id
average_item_per_order = clean_sales_2019_df.groupby('order_id')['quantity'].mean().mean()


In [ ]:
#show result as visual
with plt.style.context('classic'):
    plt.subplots(figsize=(8, 1))
    plt.axis('off')
    plt.text(0.5, 0.5,
             f'{average_item_per_order:,.2f}',
             fontsize=40, ha='center', va='center')
    plt.title('Average Item Each Order', fontsize=20)
    plt.show()


## Probability

Task 6:

Q: How much probability for next people will order certain products?

A: Find Product Sold % of Total Product Sold

The following section is to find % product sold compare to the total sold of all products.
the result will give % of product which may define the likelihood of the product will be purchased next.
This part is also the only section which used Squarify to plot the result as Treemap.

### Product Orders of Total Sales

In [ ]:
#find total product sold
total_product_sold = product_popularity_pivot['quantity'].sum()

#find sizes and labels value in product popularity
sizes = product_popularity_pivot['quantity']
labels = product_popularity_pivot.index

#calculate percentage for each product
percentages = [f'{100*sizes[i]/sizes.sum():.2f}%\n{labels[i]}' for i in range(len(sizes))]


In [ ]:
#generate treemap visual for product sold percentage of total
with plt.style.context('classic'):
    plt.figure(figsize=(20, 8))
    squarify.plot(sizes=sizes, label=percentages, alpha=0.7, color=product_color) #get color from previous plot
    plt.axis('off')
    plt.title('Product % of Total', fontsize=20)
    plt.show()


> The following will store the data as dataframe contain 'product' and 'probability'.

In [ ]:
#create empty dataframe to store probability
prob_data = []

#find percentages for product and add to data list
for p in percentages:
    prob, label = p.split('\n')
    prob = prob.replace('%', '')  #remove percent sign
    prob_data.append({'product': label, 'probability': float(prob) / 100})

#create a DataFrame from the data
probability_df = pd.DataFrame(prob_data)


> If need to show the probability as pivot table then can apply the following method.

In [ ]:
#apply background gradient to probability column
probability_pivot = pd.pivot_table(probability_df, index='product', values='probability')
probability_pivot = probability_pivot.sort_values('probability', ascending=False)
syled_probability_pivot = probability_pivot.style.background_gradient(cmap='RdYlGn', subset='probability').format({'probability': '{:.2%}'})

#pass table to html
html_probability_pivot = syled_probability_pivot.to_html()

#display tabel
display_html(f'CONCLUSION:\n'
             f'<table><tr><th>List of Probability:</th></tr><tr><td>{html_probability_pivot}</td></tr></table>', raw=True)


## Product Associate Rules

Task 7:

Q: What products are most often sold together?

A: Find Product Associate Rules using mlxtend library

First need to find the product sold by each order id and store them into series(product_by_order).
Then transform the data into one-hot encoded matrix by applying TransactionEncoder().fit_transform().
The result will be stored as dataframe(encoded_df).
Encoded data then used to generate frequent item sets using apriori algorithm the minimum support are defined as following.
Lastly sort the order by value in 'confidence'.

In [ ]:
#group product by order id
product_by_order = clean_sales_2019_df.groupby('order_id')['product'].apply(list)

#convert data to one-hot encoded matrix
te = TransactionEncoder()
onehot = te.fit_transform(product_by_order)
encoded_df = pd.DataFrame(onehot, columns=te.columns_)

#identify frequent itemsets and set parameters accordingly
frequent_itemsets_df = apriori(encoded_df, min_support=0.00000001, max_len=2, use_colnames=True)

#generate association rules and set metric to confidence(measure likelihood)
rules_df = association_rules(frequent_itemsets_df, metric='confidence', min_threshold=0.05)
rules_df = rules_df.sort_values('confidence', ascending=False)


> The following will convert antecedents and consequents from frozentset to string and show the rules as styled dataframe if needed.

In [ ]:
#apply lambda to rules_df
rules_df = rules_df.applymap(lambda x: ', '.join([str(item) for item in x]) if isinstance(x, frozenset) else x)

#show dataframe as table with gradient color in column with value
rules_df.style.background_gradient(cmap='RdYlGn', subset=['antecedent support', 'consequent support', 'support',
                                                          'confidence', 'lift', 'leverage', 'conviction'])


> Although the rules can be presented as tables.
In this case the rules is generated as string following more descriptive explanation.
There may be other method to visualize the rules waiting to be explored.
But for now the following will do.

In [ ]:
#create scatter visual
with plt.style.context('dark_background'):
    #format figure
    plt.figure(figsize=(14, 6))
    plt.scatter(x=rules_df['support'], y=rules_df['confidence'], c=rules_df['lift'], cmap='YlOrRd')
    plt.xlabel('Support')
    plt.ylabel('Confidence')
    plt.title('Association Rules', fontsize=20)

    #add colorbar
    plt.colorbar().set_label('Lift', labelpad=10)

    #add annotation
    for i, row in rules_df.iterrows():
        #find the value for antecedents, consequents, leverage, and conviction
        antecedents = row['antecedents']
        consequents = row['consequents']
        leverage = row['leverage']
        conviction = row['conviction']

        #set xy position
        x_pos = row['support']
        y_pos = row['confidence']

        #annotate formatted value
        plt.annotate(f'{antecedents}, {consequents}\nLEV: {leverage:.3f}, CONV: {conviction:.3f}',
                     xy=(x_pos, y_pos), xytext=(0, -20), textcoords='offset points', fontsize=8, ha='center')
    plt.show()


In [ ]:
#create a pivot table to format the data for heatmap
rules_pivot = pd.pivot_table(rules_df, index='antecedents', columns='consequents', values='confidence')


In [ ]:
#create heatmap visual for rules pivot table
with plt.style.context('dark_background'):
    plt.figure(figsize=(14, 6))
    sns.heatmap(data=rules_pivot, annot=True, cmap='coolwarm')
    plt.title('Confidence Map', fontsize=20)
    plt.xlabel('Consequent')
    plt.ylabel('Antecedents')
    plt.xticks(rotation=60)
    plt.yticks(rotation=0)
    plt.show()


> The following will print the top 3 rules as strings for conclusion.

In [ ]:
#create list to store items
summaries = []

#iterrate throught available items in rules (for now it is set at 3 largest confidence value)
for i, row in rules_df.nlargest(3, 'confidence').iterrows():
    antecedents = row['antecedents']
    consequents = row['consequents']
    support = row['support']
    confidence = row['confidence']
    lift = row['lift']
    leverage = row['leverage']
    conviction = row['conviction']

    #define ANSI escape codes for modifying the text's appearance
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    RED = '\033[91m'
    END = '\033[0m'

    #create summary in string
    summary = (
    f'Rule {i+1}:\n'
    f'Customers who bought {BLUE}{antecedents}{END} are more likely to buy {RED}{consequents}{END}.\n'
    f'This combination of products was purchased in {GREEN}{support*100:.3f}%{END} of all transactions.\n'
    f'If customers buy {BLUE}{antecedents}{END}, the chance of them also buying {RED}{consequents}{END} is {GREEN}{confidence*100:.2f}%{END}.\n'
    f'{RED}{consequents}{END} appears {GREEN}{lift:.2f}x{END} more likely when {BLUE}{antecedents}{END} appears.\n'
    f'The relation between {BLUE}{antecedents}{END} and {RED}{consequents}{END} is {GREEN}{leverage:.5f}{END}.\n'
    f'The rule significancy {BLUE}{antecedents}{END} and {RED}{consequents}{END} is {GREEN}{conviction:.2f}{END}.'
    )

    #append the summary
    summaries.append(summary)

#join all summaries into a single string
summary_string = '\n\n'.join(summaries)

#print summary
print(f'CONCLUSION\n\n{summary_string}')


# TO SUMMARIZE

It appears that the company generated a total of USD 34,465,537.94 in sales, with the majority of sales occurring in 2019, totaling USD 34,456,867.65. In contrast, 2020 only accounted for USD 8,670.29 in sales, which may be due to incomplete or missing data. The highest selling months were December, October, and April 2019, with sales of USD 4,608,295.70, USD 3,734,777.86 and USD 3,389,217.98 respectively, while the lowest sales occurred in January, September 2019, and January 2020, which accounted for only USD 3,915,878.85 of the total sales. The company generated the highest sales in California (CA), with San Francisco and Los Angeles accounting for USD 8,254,743.55 and USD 5,448,304.28 respectively. The lowest sales occurred in Portland, Maine, with only USD 449,321.38 in sales.

To maximize the company's advertising efforts, it is recommended to advertise in December, October, and April, as the company generated the most orders during these months (24,004, 19,436, and 17,528, respectively). Tuesdays were found to be the most profitable day for advertising, generating 26,063 orders, while Fridays had the lowest number of orders. The best times for advertising were between 11:00-13:00 and 18:00-20:00, when the company typically had the highest sales counts.

In terms of product performance, some products performed better or worse than their price indicated. The finding result is as following:
1. The Bose SoundSport Headphones had a lower price, but sold less than the Apple Airpods Headphones.
2. The 34in Ultrawide Monitor, 27in 4K Gaming Monitor, Flatscreen TV, 20in Monitor, Vareebadd Phone, LG Washing Machine, and LG Dryer all sold fewer units than the iPhone despite having lower prices.
3. In contrast, the ThinkPad Laptop and Macbook Pro Laptop performed better than expected when compared to the LG Dryer, LG Washing Machine, Vareebadd Phone, and 20in Monitor despite having higher price.
4. The 20in Monitor was the worst-performing product, selling only 4,123 units, compared to the 27in FHD Monitor, which sold 7,538 units despite having lower price.

The probability of customers ordering specific products compared to all other products was calculated as following:
1. USB-C Charging Cable: 11.46%
2. Wired Headphones: 9.83%
3. iPhone: 3.28%
4. Google Phone: 2.65%

Finally, based on rules the chance of customers who bought Google Phone are more likely to buy USB-C Charging Cable is 18.06%. While the chance of customers who bought Vareebadd Phone are more likely to buy USB-C Charging Cable is 17.83%. Lastly, the chance of customers who bought iPhone are more likely to buy Lightning Charging Cable is 14.77%. While this rules accounted for 0.5%, 0.2%, and 0.5% of total transactions respectively. Additionally, for the rest of the rules is not significant since it return with negative lift value and/or have low confidence.

# REFERENCES

Version 11:  Thanks to:

[Knightbearr](https://www.kaggle.com/knightbearr) for providing [Sales Product Data](https://www.kaggle.com/datasets/knightbearr/sales-product-data)

References at:

[Matplotlib](https://matplotlib.org/stable/api/matplotlib_configuration_api.html),
[Pandas](https://pandas.pydata.org/pandas-docs/stable/reference/frame.html),
[Squarify](https://github.com/laserson/squarify),
[Seaborn](https://seaborn.pydata.org/),
[Mlxtend](https://github.com/rasbt/mlxtend)

Inspired by:

Knightbearr - [Analysis Sales Data (Knightbearr)](https://www.kaggle.com/code/knightbearr/analysis-sales-data-knightbearr)

Peilin_C - [Sales Data Analysis with Python](https://www.kaggle.com/code/peilinc/sales-data-analysis-with-python)


# VERSION 12

1. Add function for plotting and some formatting that are reptitive.
2. Generalize some methods being used.
3. Change the majority of data selection method from dataframe to pivot table.
5. Added more markdown.

*Probably the last revision(Hopefully)